# import libraries

# IMPORTANT Install dependencies safely

Run ONLY the next cell (Install Dependencies). This cell installs required Python packages without running the pipeline.

Do NOT run other cells after installation unless you intend to re-run the full pipeline (which may regenerate CSVs or download external data).

In [1]:
# =============================================================================
# GHANA MAIZE YIELD PREDICTION PIPELINE
# =============================================================================

# 1. SETUP & IMPORTS
import os
import re
import time
import requests
import joblib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Machine Learning Libraries
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, VotingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_absolute_error
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# Utilities
from geopy.geocoders import Nominatim

# Ensure directories exist
for d in ["../data/raw", "../data/processed", "../data/final", "../models", "../reports"]:
    os.makedirs(d, exist_ok=True)

print("✅ Environment Ready.")

✅ Environment Ready.


# helper functiions

In [2]:


# =============================================================================
# 2. HELPER FUNCTIONS
# =============================================================================
def clean_district_name(name):
    """Standardizes district names for consistent merging."""
    if pd.isna(name): return ""
    name = str(name).lower()
    name = re.sub(r'\s*\([^)]*\)', '', name)  # Remove content in brackets
    name = name.replace(' municipal', '').replace(' muni', '').replace(' district', '')
    name = name.replace(' metropolitan', '').replace(' metro', '')
    name = name.replace('-', ' ')
    name = re.sub(r'\s+', ' ', name).strip()
    return name.title()

def get_soil_map():
    """Maps Administrative Regions to Dominant Soil Types."""
    return {
        'Ashanti': 'Forest Ochrosol', 'Brong Ahafo': 'Forest Ochrosol',
        'Bono': 'Forest Ochrosol', 'Bono East': 'Forest Ochrosol',
        'Ahafo': 'Forest Ochrosol', 'Eastern': 'Forest Ochrosol',
        'Western': 'Forest Ochrosol', 'Western North': 'Forest Ochrosol',
        'Central': 'Forest Ochrosol', 'Northern': 'Savanna Ochrosol',
        'Savannah': 'Savanna Ochrosol', 'North East': 'Savanna Ochrosol',
        'Upper East': 'Savanna Ochrosol', 'Upper West': 'Savanna Ochrosol',
        'Oti': 'Savanna Ochrosol', 'Greater Accra': 'Coastal Savannah',
        'Volta': 'Tropical Black Earth'
    }





# data processing

In [4]:
# =============================================================================
# 3. DATA PROCESSING
# =============================================================================
print("\n--- PHASE 1: DATA INGESTION ---")

# A. LOAD RAW YIELD DATA
# We look for the file in data/raw or the root folder
raw_path = "../data/raw/maize_yield.csv"
if not os.path.exists(raw_path) and os.path.exists("maize_yield.csv"):
    raw_path = "maize_yield.csv" # Fallback for Colab root

if os.path.exists(raw_path):
    df_yield = pd.read_csv(raw_path)
    # Clean Columns
    df_yield.columns = [c.strip().title() for c in df_yield.columns]
    
    # Identify District Column dynamically
    dist_col = next((c for c in df_yield.columns if 'District' in c or 'DISTRICT' in c), None)
    if dist_col:
        df_yield.rename(columns={dist_col: 'District', 'YEAR': 'Year', 'YIELD': 'Yield'}, inplace=True)
        df_yield['District'] = df_yield['District'].apply(clean_district_name)
        print(f"✅ Yield Data Loaded: {len(df_yield)} rows (2010-2016)")
    else:
        raise ValueError("Could not find 'District' column in CSV.")
else:
    raise FileNotFoundError("❌ 'maize_yield.csv' not found. Please upload it to data/raw/")

# B. IMPUTE MISSING YEARS (2017-2021)
# We generate new rows using National Averages to capture the PFJ Policy era
print("   -> Imputing 2017-2021 data using National Averages...")
unique_districts = df_yield['District'].unique()
national_avgs = [
    {'Year': 2017, 'Yield': 2.04}, {'Year': 2018, 'Yield': 2.25},
    {'Year': 2019, 'Yield': 2.53}, {'Year': 2020, 'Yield': 2.58},
    {'Year': 2021, 'Yield': 2.53}
]
new_rows = [{'District': d, 'Year': entry['Year'], 'Yield': entry['Yield']} 
            for d in unique_districts for entry in national_avgs]
df_extended = pd.concat([df_yield[['District', 'Year', 'Yield']], pd.DataFrame(new_rows)], ignore_index=True)

# C. GET WEATHER DATA (NASA API)
weather_path = "../data/processed/Step2_Weather_Data.csv"

if os.path.exists(weather_path):
    df_weather = pd.read_csv(weather_path)
    print("✅ Weather Data Loaded from cache.")
else:
    print("⚠️ Weather file not found. Downloading from NASA (This takes ~5 mins)...")
    geolocator = Nominatim(user_agent="ghana_maize_project_v1")
    weather_data = []
    base_url = "https://power.larc.nasa.gov/api/temporal/daily/point"
    
    for district in unique_districts:
        try:
            # Geolocate
            loc = geolocator.geocode(f"{district}, Ghana", timeout=10)
            if loc:
                # API Request
                params = {
                    "parameters": "T2M,PRECTOTCORR,RH2M,ALLSKY_SFC_SW_DWN,GWETTOP",
                    "community": "AG", "longitude": loc.longitude, "latitude": loc.latitude,
                    "start": "20100101", "end": "20211231", "format": "JSON"
                }
                resp = requests.get(base_url, params=params)
                
                if resp.status_code == 200:
                    d = resp.json()['properties']['parameter']
                    temp_df = pd.DataFrame({
                        'Date': pd.to_datetime(list(d['T2M'].keys())),
                        'Rainfall': list(d['PRECTOTCORR'].values()),
                        'Temperature': list(d['T2M'].values()),
                        'Humidity': list(d['RH2M'].values()),
                        'Sunlight': list(d['ALLSKY_SFC_SW_DWN'].values()),
                        'Soil_Moisture': list(d['GWETTOP'].values()),
                        'District': district
                    })
                    weather_data.append(temp_df)
            time.sleep(1) # Be polite to API
        except Exception as e:
            print(f"Error for {district}: {e}")
    
    if weather_data:
        df_weather = pd.concat(weather_data)
        # Ensure output directory exists then save to the intended path
        os.makedirs(os.path.dirname(weather_path), exist_ok=True)
        df_weather.to_csv(weather_path, index=False)
        print("✅ Weather Download Complete.")
    else:
        raise ValueError("❌ Failed to download weather data.")

# Aggregating Weather (Seasonal: April - August)
df_weather['Date'] = pd.to_datetime(df_weather['Date'])
df_weather['Month'] = df_weather['Date'].dt.month
df_weather['Year'] = df_weather['Date'].dt.year

season_weather = df_weather[df_weather['Month'].isin([4, 5, 6, 7, 8])]
weather_agg = season_weather.groupby(['District', 'Year']).agg({
    'Rainfall': 'sum',
    'Temperature': 'mean',
    'Humidity': 'mean',
    'Sunlight': 'mean',
    'Soil_Moisture': 'mean'
}).reset_index()

# D. ADD SOIL DATA
# If original file had Region, we map it. If not, we default to Forest Ochrosol.
if 'Region' in df_yield.columns:
    df_soil = df_yield[['District', 'Region']].drop_duplicates()
    df_soil['Region'] = df_soil['Region'].str.title().str.strip()
    df_soil['Soil_Type'] = df_soil['Region'].map(get_soil_map()).fillna('Forest Ochrosol')
else:
    df_soil = pd.DataFrame({'District': unique_districts})
    df_soil['Soil_Type'] = 'Forest Ochrosol'


--- PHASE 1: DATA INGESTION ---
✅ Yield Data Loaded: 1681 rows (2010-2016)
   -> Imputing 2017-2021 data using National Averages...
⚠️ Weather file not found. Downloading from NASA (This takes ~5 mins)...
✅ Weather Download Complete.


# feature engineering

In [5]:
# =============================================================================
# 4. FEATURE ENGINEERING
# =============================================================================
print("\n--- PHASE 2: ENGINEERING ---")

# Merge
master_df = pd.merge(df_extended, weather_agg, on=['District', 'Year'], how='inner')
master_df = pd.merge(master_df, df_soil[['District', 'Soil_Type']], on='District', how='left')

# 1. Winsorization (Cap Outliers)
master_df['Yield'] = np.where(master_df['Yield'] > 4.0, 4.0, master_df['Yield'])

# 2. Add Event Shocks
master_df['Pest_Risk'] = np.where(master_df['Year'].isin([2016, 2017]), 1, 0)
master_df['PFJ_Policy'] = np.where(master_df['Year'] >= 2017, 1, 0)

# 3. Add Lag Features (Crucial Step)
master_df.sort_values(['District', 'Year'], inplace=True)
master_df['Yield_Lag1'] = master_df.groupby('District')['Yield'].shift(1)

# Remove rows with NaN (First year has no history)
final_df = master_df.dropna().reset_index(drop=True)

# Save Master Dataset
final_path = "../data/final/Ghana_Maize_Master_Dataset.csv"
final_df.to_csv(final_path, index=False)
print(f"✅ Master Dataset Created: {final_df.shape}")




--- PHASE 2: ENGINEERING ---
✅ Master Dataset Created: (1775, 12)


# model

In [6]:
# =============================================================================
# 5. MODEL TRAINING
# =============================================================================
print("\n--- PHASE 3: TRAINING ENSEMBLE ---")

X = final_df[['Rainfall', 'Temperature', 'Humidity', 'Sunlight', 'Soil_Moisture', 
              'Soil_Type', 'Pest_Risk', 'PFJ_Policy', 'Year', 'Yield_Lag1']]
y = final_df['Yield']

# Pipeline Setup
preprocessor = ColumnTransformer(transformers=[
    ('cat', OneHotEncoder(handle_unknown='ignore'), ['Soil_Type']),
    ('num', 'passthrough', [c for c in X.columns if c != 'Soil_Type'])
])

# Voting Ensemble (Random Forest + Gradient Boosting)
ensemble = VotingRegressor([
    ('rf', RandomForestRegressor(n_estimators=200, random_state=42)),
    ('gb', GradientBoostingRegressor(n_estimators=200, random_state=42))
])

model_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('regressor', ensemble)
])

# Train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
model_pipeline.fit(X_train, y_train)

# Evaluate
y_pred = model_pipeline.predict(X_test)
r2 = r2_score(y_test, y_pred)
mae = mean_absolute_error(y_test, y_pred)

print(f"🚀 Model Accuracy (R2): {r2:.3f}")
print(f"📉 Error Margin (MAE):  {mae:.3f} tons/ha")

# Save Model
joblib.dump(model_pipeline, "../models/maize_model.pkl")
print("💾 Model saved to 'models/maize_model.pkl'")


--- PHASE 3: TRAINING ENSEMBLE ---
🚀 Model Accuracy (R2): 0.899
📉 Error Margin (MAE):  0.078 tons/ha
💾 Model saved to 'models/maize_model.pkl'


In [11]:
# =============================================================================
# 6. VISUALIZATIONS: Save plots to ../reports
# =============================================================================
import os
os.makedirs("../reports", exist_ok=True)
# Set plotting style robustly (fallbacks if style name not available)
try:
    plt.style.use('seaborn-darkgrid')
except Exception:
    try:
        sns.set_theme(style='darkgrid')
    except Exception:
        try:
            sns.set_style('darkgrid')
        except Exception:
            plt.style.use('ggplot')

# 1) National average yield trend by Year
yr_df = final_df.groupby('Year')['Yield'].mean().reset_index()
plt.figure(figsize=(8,4))
sns.lineplot(data=yr_df, x='Year', y='Yield', marker='o')
plt.title('National Average Maize Yield by Year')
plt.ylabel('Yield (tons/ha)')
plt.xlabel('Year')
plt.savefig('../reports/national_yield_trend.png', bbox_inches='tight')
plt.close()

# 2) Yield distribution by Soil Type (boxplot)
plt.figure(figsize=(10,6))
sns.boxplot(data=final_df, x='Soil_Type', y='Yield')
plt.xticks(rotation=45, ha='right')
plt.title('Yield Distribution by Soil Type')
plt.xlabel('Soil Type')
plt.ylabel('Yield (tons/ha)')
plt.savefig('../reports/yield_by_soil_type.png', bbox_inches='tight')
plt.close()

# 3) Rainfall (Apr-Aug) vs Yield with regression line
plt.figure(figsize=(8,6))
sns.regplot(data=final_df, x='Rainfall', y='Yield', scatter_kws={'s':10}, line_kws={'color':'red'})
plt.title('Rainfall (Apr-Aug) vs Yield')
plt.xlabel('Rainfall (mm)')
plt.ylabel('Yield (tons/ha)')
plt.savefig('../reports/rainfall_vs_yield.png', bbox_inches='tight')
plt.close()

# 4) Correlation heatmap among key numeric features
num_cols = ['Yield','Rainfall','Temperature','Humidity','Sunlight','Soil_Moisture','Yield_Lag1']
corr = final_df[num_cols].corr()
plt.figure(figsize=(8,6))
sns.heatmap(corr, annot=True, fmt='.2f', cmap='coolwarm')
plt.title('Feature Correlation Matrix')
plt.savefig('../reports/feature_correlation.png', bbox_inches='tight')
plt.close()

# 5) Feature importances (attempt to extract from RandomForest in ensemble)
try:
    reg = model_pipeline.named_steps['regressor']
    rf = None
    if hasattr(reg, 'estimators_') and len(reg.estimators_)>0:
        candidate = reg.estimators_[0]
        if isinstance(candidate, tuple):
            rf = candidate[1]
        else:
            rf = candidate

    if rf is not None and hasattr(rf, 'feature_importances_'):
        importances = rf.feature_importances_
        pre = model_pipeline.named_steps['preprocessor']
        num_features = [c for c in X.columns if c != 'Soil_Type']
        try:
            ohe = pre.named_transformers_['cat']
            ohe_features = list(ohe.get_feature_names_out(['Soil_Type']))
        except Exception:
            ohe_features = []
        feature_names = num_features + ohe_features
        if len(importances) != len(feature_names):
            feature_names = [f'F{i}' for i in range(len(importances))]
        feat_imp_df = pd.DataFrame({'feature': feature_names, 'importance': importances}).sort_values('importance', ascending=False)
        plt.figure(figsize=(8,6))
        # Assign hue to 'feature' (avoids palette-without-hue deprecation) and remove legend
        ax = sns.barplot(data=feat_imp_df, x='importance', y='feature', hue='feature', dodge=False, palette='viridis')
        try:
            lg = ax.get_legend()
            if lg is not None:
                lg.remove()
        except Exception:
            pass
        plt.title('Feature Importances (Random Forest)')
        plt.xlabel('Importance')
        plt.ylabel('Feature')
        plt.savefig('../reports/feature_importances.png', bbox_inches='tight')
        plt.close()
    else:
        print('RandomForest feature importances not available.')
except Exception as e:
    print('Could not compute feature importances:', e)

print('✅ Visualizations saved to ../reports/')

✅ Visualizations saved to ../reports/
